In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px 
plt.style.use('ggplot')

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from scipy.stats import norm
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from IPython.core.display import HTML

%matplotlib inline

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

In [2]:
# read in dataset that calculated scores on 8 hours within derate 
codes = pd.read_csv('data/train_codes_2hr.csv', low_memory=False)

codes.head()

,Unnamed: 0,active_transition_count,accelerator_pedal,barometric_pressure,cruise_control_active,cruise_control_set_speed,distance_ltd,engine_coolant_temperature,engine_load,engine_oil_pressure,engine_oil_temperature,engine_rpm,engine_time_ltd,fuel_level,fuel_ltd,fuel_rate,fuel_temperature,ign_status,intake_manifold_temperature,parking_brake,speed,switched_battery_voltage,throttle,turbo_boost_pressure,target,spn_0,spn_100,spn_101,spn_102,spn_1023,spn_1024,spn_1028,spn_103,spn_1043,spn_1045,spn_105,spn_1056,spn_1059,spn_1067,spn_1068,spn_107,spn_1071,spn_1072,spn_1075,spn_1078,spn_108,spn_1081,spn_110,spn_111,spn_1127,spn_114863,spn_116,spn_1172,spn_1176,spn_118,spn_1209,spn_1213,spn_1231,spn_1235,spn_1236,spn_1239,spn_1247,spn_125,spn_127,spn_1321,spn_1322,spn_1323,spn_1324,spn_1325,spn_1326,spn_1327,spn_1328,spn_1347,spn_1349,spn_13600,spn_139296,spn_1464,spn_1481,spn_1482,spn_1483,spn_1487,spn_153,spn_153931,spn_1569,spn_157,spn_158,spn_16,spn_160,spn_1612,spn_1659,spn_1668,spn_167,spn_1675,spn_168,spn_17096,spn_171,spn_173,spn_174,spn_175,spn_17590,spn_1761,spn_177,spn_1787,spn_1807,spn_1808,spn_1809,spn_1815,spn_184,spn_188,spn_190,spn_191,spn_196608,spn_2000,spn_2017,spn_2023,spn_2029,spn_228,spn_235,spn_236,spn_237,spn_245,spn_247,spn_248,spn_251,spn_252,spn_255,spn_256,spn_25780,spn_2579,spn_2623,spn_2629,spn_2630,spn_2659,spn_27,spn_2791,spn_2795,spn_2863,spn_2866,spn_2912,spn_2917,spn_29902,spn_3031,spn_3058,spn_3060,spn_3064,spn_32000,spn_3216,spn_3217,spn_3218,spn_3222,spn_3226,spn_3227,spn_3228,spn_3241,spn_3242,spn_3245,spn_3246,spn_3249,spn_3251,spn_3253,spn_33,spn_335040,spn_3360,spn_3361,spn_3362,spn_3363,spn_3364,spn_3464,spn_3480,spn_3482,spn_3490,spn_3509,spn_3510,spn_3511,spn_3512,spn_3513,spn_3515,spn_3521,spn_35527,spn_3556,spn_3583,spn_3597,spn_36017,spn_3605,spn_3610,spn_3663,spn_3695,spn_3696,spn_3697,spn_3698,spn_37,spn_3703,spn_3720,spn_38,spn_3821,spn_39093,spn_3936,spn_4094,spn_4095,spn_4096,spn_411,spn_412,spn_4219,spn_4220,spn_4276,spn_43088,spn_4331,spn_4334,spn_4339,spn_4340,spn_4342,spn_4344,spn_4346,spn_4349,spn_4354,spn_4356,spn_4360,spn_4363,spn_4364,spn_4375,spn_4376,spn_4380,spn_4382,spn_441,spn_442,spn_444,spn_47284,spn_4752,spn_4765,spn_4766,spn_4792,spn_4794,spn_4795,spn_4796,spn_4811,spn_4812,spn_4813,spn_5018,spn_5019,spn_5024,spn_5031,spn_50353,spn_5052,spn_51,spn_5109,spn_5110,spn_5111,spn_5112,spn_5113,spn_5114,...,spn_520413,spn_520953,spn_521032,spn_522,spn_523530,spn_523531,spn_523543,spn_524033,spn_524037,spn_524071,spn_524287,spn_5245,spn_525,spn_5298,spn_5319,spn_5321,spn_5357,spn_5392,spn_5394,spn_5395,spn_5396,spn_5397,spn_5442,spn_5443,spn_5444,spn_5485,spn_5491,spn_5569,spn_5571,spn_5579,spn_558,spn_5585,spn_560,spn_5614,spn_5615,spn_5616,spn_5625,spn_563,spn_5742,spn_5743,spn_5745,spn_5746,spn_576,spn_577,spn_578,spn_583,spn_5835,spn_5848,spn_5851,spn_5853,spn_5862,spn_5902,spn_5903,spn_5909,spn_5939,spn_5941,spn_5942,spn_5953,spn_596,spn_603,spn_609,spn_611,spn_612,spn_614,spn_6145,spn_6146,spn_6147,spn_6148,spn_624,spn_627,spn_628,spn_629,spn_630,spn_632,spn_633,spn_636,spn_639,spn_641,spn_647,spn_649,spn_651,spn_652,spn_653,spn_65302,spn_65303,spn_654,spn_655,spn_65535,spn_656,spn_6713,spn_6773,spn_6780,spn_6802,spn_70,spn_705,spn_709,spn_723,spn_729,spn_7321,spn_7323,spn_74,spn_75,spn_751,spn_752,spn_76339,spn_767,spn_768,spn_77,spn_78,spn_781,spn_78132,spn_7827,spn_7847,spn_7854,spn_788,spn_789,spn_790,spn_791,spn_792,spn_793,spn_794,spn_795,spn_796,spn_797,spn_798,spn_799,spn_800,spn_801,spn_802,spn_803,spn_805,spn_806,spn_807,spn_81,spn_810,spn_829,spn_830,spn_84,spn_862,spn_88121,spn_886,spn_904,spn_905,spn_906,spn_907,spn_91,spn_917,spn_92,spn_929,spn_9295,spn_932,spn_933,spn_934,spn_937,spn_938,spn_939,spn_94,spn_940,spn_941,spn_95,spn_96,spn_97,spn_976,spn_98,fmi_0,fmi_1,fmi_2,fmi_3,fmi_4,fmi_5,fmi_6,fmi_7,fmi_8,fmi_9,fmi_10,fmi_11,fmi_12,fmi_13,fmi_14,fmi_15,fmi_16,fmi_17,fmi_18,fmi_19,fmi_20,fmi_21,fmi_22,fmi_23,fmi_29,fmi_31,lamp_status_0,lamp_status_2,lamp

In [3]:
codes = codes.drop(columns=['Unnamed: 0'])

In [4]:
#Define predictors
X = codes.drop('target', axis=1).values

#Define target
y = codes['target']


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state = 20, stratify=y)



In [6]:
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [7]:
oversampler = SMOTE(k_neighbors=10, n_jobs=-1, random_state=42)

In [8]:
X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

## 2-hr window, Random Forest 

In [9]:
rf = RandomForestClassifier(n_estimators=150, max_depth=2, random_state=0).fit(X_smote, y_smote)

In [10]:
y_pred = rf.predict(X_test)

In [11]:
accuracy_score(y_test, y_pred)

0.9248697064729373

In [12]:
confusion_matrix(y_test, y_pred)

array([[122944,   9975],
       [    15,     35]], dtype=int64)

In [13]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.92      0.96    132919
           1       0.00      0.70      0.01        50

    accuracy                           0.92    132969
   macro avg       0.50      0.81      0.48    132969
weighted avg       1.00      0.92      0.96    132969



In [ ]:
#For some reason the code below is erring out.  It worked on the previous run when we did not use SMOTE

In [21]:
# feature_importances = pd.DataFrame({'feature': X.columns, 'importance': rf.feature_importances_})

# feature_importances.sort_values('importance', ascending = False).head(10)

# This is code to refine the hyperparameters using randomized search cv; however, I had to terminate the kernel after a couple of hours of running

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#refence:  towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [ ]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 25, cv = 3, verbose=2, random_state=42, n_jobs = -1)



In [ ]:
# Fit the random search model.  This took hours! 
rf_random.fit(X_smote, y_smote) 

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.best_score_

Reference: https://medium.com/@alexstrebeck/training-and-testing-machine-learning-models-e1f27dc9b3cb

https://towardsdatascience.com/machine-learning-with-datetime-feature-engineering-predicting-healthcare-appointment-no-shows-5e4ca3a85f96

https://machinelearningmastery.com/precision-recall-and-f-measure-for-imbalanced-classification/

https://medium.com/making-sense-of-data/time-series-next-value-prediction-using-regression-over-a-rolling-window-228f0acae363